<a href="https://colab.research.google.com/github/shubhamgautam1211/Machine-Learning-Standalone/blob/main/SG_21_KNN_BookRecommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Book Recommendation system**

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from scipy.sparse import csr_matrix
import seaborn
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

In [ ]:
books = pd.read_csv("BX-Books.csv", sep=';', encoding="latin-1", error_bad_lines=False)
users = pd.read_csv("BX-Users.csv", sep=';', encoding="latin-1", error_bad_lines=False)
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=';', encoding="latin-1", error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:27

In [ ]:
books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


**drop urls**

In [ ]:
books.drop(['Image-URL-S','Image-URL-M','Image-URL-L'],axis=1,inplace=True)

In [ ]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)
users.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)
ratings.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

In [ ]:
books.head(5)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
users.head(5)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
print(books.nunique())
print("---------------------------------------")
print(users.nunique())
print("---------------------------------------")
print(ratings.nunique())

ISBN         271360
title        242135
author       102023
year            202
publisher     16807
dtype: int64
---------------------------------------
user_id     278858
location     57339
age            165
dtype: int64
---------------------------------------
user_id    105283
ISBN       340556
rating         11
dtype: int64


In [ ]:
print(books.isnull().sum())
print("---------------------------------------")
print(users.isnull().sum())
print("---------------------------------------")
print(ratings.isnull().sum())

ISBN         0
title        0
author       1
year         0
publisher    2
dtype: int64
---------------------------------------
user_id          0
location         0
age         110762
dtype: int64
---------------------------------------
user_id    0
ISBN       0
rating     0
dtype: int64


In [ ]:
users.age.fillna(round(users.age.mean()), inplace = True)

In [ ]:
print(users.isnull().sum())

user_id     0
location    0
age         0
dtype: int64


Create a new dataframe “Combine_Book_Ratings by merging books csv file 
with their ratings csv file using pd.merge( )


In [ ]:
Combine_Book_Ratings = ratings.merge(books, on='ISBN')
Combine_Book_Ratings.head()

,user_id,ISBN,rating,title,author,year,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


Calculate total rating count for the books based on the book_ratings columns using groupby( ) function.

In [ ]:
total_rating = Combine_Book_Ratings.groupby('title')['rating'].count().reset_index()

total_rating.rename(columns= {'rating':'total_rating'}, inplace=True)
total_rating = Combine_Book_Ratings.merge(total_rating, on='title')
total_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,total_rating
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60


Filter out the lesser known books using total rating count. You need to take only those books for which total rating count is greater than or equal to 50.

In [ ]:
total_rating = total_rating[total_rating['total_rating'] >= 50]
total_rating.drop_duplicates(['user_id','title'], inplace=True)

total_rating.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(285518, 8)

Also filter the users based on the location. Take into consideration the users of “usa” and “Canada”

In [ ]:
combined = total_rating.merge(users, left_on = 'user_id', right_on = 'user_id', how = 'left')

us_canada_user_rating = combined[combined['location'].str.contains("usa|canada")]
us_canada_user_rating=us_canada_user_rating.drop(['age'],axis=1) #not useful
us_canada_user_rating.head(10)

,user_id,ISBN,rating,title,author,year,publisher,total_rating,location
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"cincinnati, ohio, usa"
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"strafford, missouri, usa"
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"st. charles county, missouri, usa"
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"beaverton, oregon, usa"
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"st. louis, missouri, usa"
6,28266,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"portland, oregon, usa"
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"springfield, missouri, usa"
9,50403,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"conway, arkansas, usa"
10,56157,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60,"florissant, missouri, usa"


In [ ]:
us_canada_user_rating.shape

(248949, 9)

Convert your table into a 2D matrix using userid and movie title and fill the values with movie ratings

In [ ]:
book_pivot = us_canada_user_rating.pivot_table(columns='user_id', index='title', values="rating")
book_pivot.fillna(0, inplace=True)

book_pivot.shape

(2442, 40017)

Then transform the values(ratings) of the matrix dataframe into a scipy sparse matrix for more efficient calculations

In [ ]:
book_sparse_matrix = csr_matrix(book_pivot.values)
book_sparse_matrix

<2442x40017 sparse matrix of type '<class 'numpy.float64'>'
	with 97607 stored elements in Compressed Sparse Row format>

-> Define model using sklearn.neighbors to find out 5 most similar books for the given book based on the title

-> Test your model using by giving any random book_title and see the books suggestions
returned by your model.

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric = 'cosine',algorithm='brute')
model.fit(book_sparse_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[1000, :].values.reshape(1, -1))

**5 most similar books based on title**

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Legacy of Silence', 'A Secret Affair', 'Promises', 'Lovers',
       'After the Fire'],
      dtype='object', name='title')


**suggestions returned**

In [ ]:
#query_index=500
query_index = np.random.choice(book_pivot.shape[0])
distances,indices=model.kneighbors(book_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)
 
for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendation for {0}:\n".format(book_pivot.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}:".format(i,book_pivot.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendation for Tom Clancy's Op-Center Balance of Power (Tom Clancy's Op Center (Paperback)):

1: Tom Clancy's Op-Center: Games of State (Tom Clancy's Op Center (Paperback)), with distance of 0.7158479066867228:
2: State of Siege (Tom Clancy's Op-Center, 6), with distance of 0.831343990627428:
3: Breaking Point (Tom Clancy's Net Force, No. 4), with distance of 0.8400194079665709:
4: Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback)), with distance of 0.8478538318530834:
5: The Great Train Robbery, with distance of 0.854048723376844:
